In [137]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
import nltk
import gensim
from gensim import corpora

In [233]:
app = pd.read_pickle('app_cleaned.pickle')

In [234]:
app.columns

Index([u'category', u'current_rating', u'description', u'id',
       u'is_InAppPurcased', u'is_multilingual', u'is_multiplatform', u'name',
       u'new_version_desc', u'num_current_rating', u'num_overall_rating',
       u'overall_rating', u'price', u'publish_date', u'review1',
       u'review1_star', u'review2', u'review2_star', u'review3',
       u'review3_star', u'scrape_date', u'seller', u'size', u'update_date',
       u'url', u'version'],
      dtype='object')

Simply use current rating to define the quality of a app. If the current rating is no less than 4.0, it can be seen as a good app. If the current rating is no more than 2.5, it is a bad app.

In [719]:
good_app = app.loc[app['current_rating'] >=4.0]
bad_app = app.loc[app['current_rating'] <=2.5]
good_app = good_app.reset_index(drop=True)
bad_app = bad_app.reset_index(drop=True)


In [722]:
category = app['category']
cate_list = []
for i in category.unique():
    cate = i.lower()
    cate_list.append(cate)

In [243]:
#len(app[pd.isnull(app['current_rating'])==True])

1136

In [246]:
#bad_des = map(lambda x: x.decode("utf8").replace(u"\u2022", "*").replace(u"\u2019", "'").replace(u"\xa0", ' ').replace(u"\u2014", '--').replace(u"\u2013", '-'),bad_app['description'])
#good_des = map(lambda x: x.decode("utf8").replace(u"\u2022", "*").replace(u"\u2019", "'").replace(u"\xa0", ' ').replace(u"\u2014", '--').replace(u"\u2013", '-'),good_app['description'])

Use star value of different reviews to filter comments. 

In [285]:
good_rev1 = good_app.loc[good_app['review1_star']>=4].reset_index(drop=True)['review1']
good_rev2 = good_app.loc[good_app['review2_star']>=4].reset_index(drop=True)['review2']
good_rev3 = good_app.loc[good_app['review3_star']>=4].reset_index(drop=True)['review3']
bad_rev1 = bad_app.loc[bad_app['review1_star']<=2.5].reset_index(drop=True)['review1']
bad_rev2 = bad_app.loc[bad_app['review2_star']<=2.5].reset_index(drop=True)['review2']
bad_rev3 = bad_app.loc[bad_app['review3_star']<=2.5].reset_index(drop=True)['review3']

In [621]:
good_rev = good_rev1.append(good_rev2)
all_good = rev.append(good_rev3)
bad_rev = bad_rev1.append(bad_rev2)
all_bad = bad_rev.append(bad_rev3)

In [643]:
first_good = all_good[0:len(all_good)/3]
second_good = all_good[len(all_good)/3:len(all_good)*2/3]
third_good = all_good[len(all_good)*2/3:len(all_good)]
first_bad= all_bad[0:len(all_bad)/3]
second_bad = all_bad[len(all_bad)/3:len(all_bad)*2/3]
third_bad = all_bad[len(all_bad)*2/3:len(all_bad)]

<b>Cleaning and Preprocessing</b>

Data cleaning is absolutely crucial for generating a useful topic model. The steps below are common to most natural language processing methods:
   * Tokenizing: converting a document to its atomic elements.
   * Stopping: removing meaningless words.
   * Stemming: merging words that are equivalent in meaning.

Here we need to note that POS tag filter is more about the context of the features than frequencies of features. Topic Modelling tries to map out the recurring patterns of terms into topics. However, every term might not be equally important contextually. For example, POS tag IN contain terms such as – “within”, “upon”, “except”. “CD” contains – “one”,”two”, “hundred” etc. “MD” contains “may”, “must” etc. These terms are the supporting words of a language and can be removed by studying their post tags.

In [729]:
stop = set(stopwords.words('english')+[u'one',u'app',u'it',u'dont',u"i",u"'s","''","``",u'use',u'used',u'using',u'love',
                                      u'would',u'great',u'app.',u'like',u'lot']+ cate_list)
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    tokenize = nltk.word_tokenize
    to_token = stem(tokenize(normalized))
    tags = nltk.pos_tag(to_token)
    dt_tags = [t for t in tags if t[1] in ["DT", "MD", "VBP","IN", "JJ","VB"]]
    for tag in dt_tags:
        normalized = " ".join(tok for tok in to_token if tok not in tag[0])
    return normalized

In [747]:
doc_clean_g1 = [clean(doc).split() for doc in first_good]
doc_clean_g2 = [clean(doc).split() for doc in second_good]
doc_clean_g3 = [clean(doc).split() for doc in third_good]
doc_clean_b1 = [clean(doc).split() for doc in first_bad]
doc_clean_b2 = [clean(doc).split() for doc in second_bad]
doc_clean_b3 = [clean(doc).split() for doc in third_bad]

In [748]:
doc_clean_g3

[[u'best',
  u'mani',
  u'trip',
  u'friend',
  u'trip',
  u'end',
  u'enabl',
  u'simplifi',
  u'group',
  u'debt',
  u'voila',
  u'settl',
  u'keep',
  u'simpl',
  u'instead',
  u'figur',
  u'math'],
 [u'exactli',
  u'credit',
  u'bureau',
  u'experi',
  u'two',
  u'major',
  u'alway',
  u'mediocr',
  u'pleas',
  u'experian',
  u'least',
  u'allow',
  u'see',
  u'real',
  u'inform',
  u'disput',
  u'seamlessli',
  u'also',
  u'fan',
  u'amount',
  u'fee',
  u'membership'],
 [u'complaint',
  u'well',
  u'big',
  u'issu',
  u'ye',
  u'hate',
  u'pay',
  u'10',
  u'buck',
  u'take',
  u'payment',
  u'faster',
  u'standard',
  u'besid',
  u'im',
  u'ok',
  u'card',
  u'bad',
  u'credit',
  u'put',
  u'faith',
  u'anyway',
  u'gave',
  u'card',
  u'ye',
  u'interest',
  u'high',
  u'gave',
  u'card',
  u'els',
  u'cant',
  u'cancel',
  u'well',
  u'show',
  u'credit',
  u'report',
  u'card',
  u'year',
  u'happi',
  u'servic',
  u'receiv',
  u'even',
  u'help',
  u'fraud',
  u'depart',
  

<b>Preparing Document-Term Matrix</b>
* Convert a corpus into a document-term matrix. LDA model looks for repeating term patterns in the entire DT matrix

In [749]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index.
dictionary_g1 = corpora.Dictionary(doc_clean_g1)
dictionary_g2 = corpora.Dictionary(doc_clean_g2)
dictionary_g3 = corpora.Dictionary(doc_clean_g3)
dictionary_b1 = corpora.Dictionary(doc_clean_b1)
dictionary_b2 = corpora.Dictionary(doc_clean_b2)
dictionary_b3 = corpora.Dictionary(doc_clean_b3)

In [ ]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix_g1 = [dictionary.doc2bow(doc) for doc in doc_clean_g1]
doc_term_matrix_g2 = [dictionary.doc2bow(doc) for doc in doc_clean_g2]
doc_term_matrix_g3 = [dictionary.doc2bow(doc) for doc in doc_clean_g3]
doc_term_matrix_b1 = [dictionary.doc2bow(doc) for doc in doc_clean_b1]
doc_term_matrix_b2 = [dictionary.doc2bow(doc) for doc in doc_clean_b2]
doc_term_matrix_b3 = [dictionary.doc2bow(doc) for doc in doc_clean_b3]

<b>Running LDA Model (Batch Wise LDA)</b>
   * In order to retrieve most important topic terms, a corpus can be divided into batches of fixed sizes. Running LDA multiple times on these batches will provide different results, however, the best topic terms will be the intersection of all batches.


In [750]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

In [762]:
# Running and Trainign LDA model on the document term matrix.
ldamodel_g1 = Lda(doc_term_matrix_g1, num_topics=3, id2word = dictionary, passes=50)
ldamodel_g2 = Lda(doc_term_matrix_g2, num_topics=3, id2word = dictionary, passes=50)
ldamodel_g3 = Lda(doc_term_matrix_g3, num_topics=3, id2word = dictionary, passes=50)
ldamodel_b1 = Lda(doc_term_matrix_b1, num_topics=3, id2word = dictionary, passes=50)
ldamodel_b2 = Lda(doc_term_matrix_b2, num_topics=3, id2word = dictionary, passes=50)
ldamodel_b3 = Lda(doc_term_matrix_b3, num_topics=3, id2word = dictionary, passes=50)

<b>Examining the results</b>

In [766]:
print(ldamodel_g1.print_topics(num_topics=3, num_words=5))
print(ldamodel_g2.print_topics(num_topics=3, num_words=5))
print(ldamodel_g3.print_topics(num_topics=3, num_words=5))

[(0, u'0.024*5 + 0.024*free + 0.023*option + 0.022*need + 0.022*featur'), (1, u'0.033*inform + 0.032*check + 0.024*app + 0.023*detail + 0.022*truli'), (2, u'0.016*time + 0.012*show + 0.010*choos + 0.009*get + 0.009*realli')]
[(0, u'0.013*get + 0.013*time + 0.011*read + 0.010*im + 0.009*pleas'), (1, u'0.034*video + 0.031*find + 0.022*inform + 0.022*often + 0.022*daili'), (2, u'0.021*thank + 0.021*time + 0.021*speed + 0.021*hous + 0.016*track')]
[(0, u'0.027*get + 0.019*use + 0.019*app + 0.018*color + 0.018*inform'), (1, u'0.027*screen + 0.020*know + 0.019*also + 0.019*right + 0.019*need'), (2, u'0.017*free + 0.016*call + 0.013*phone + 0.012*make + 0.012*work')]


Each generated topic is separated by a comma. Within each topic are the five most probable words to appear in that topic. The best topic terms will be the intersection of all three batches. Some things to think about, for the good app, the comments have common features like: 
  1. It's free and have some good features.
  2. It has many good information and details, and customers are comfortable at vision, including video, color and screen.
  3. The speed is awesome and save some time.



In [764]:
print(ldamodel_b1.print_topics(num_topics=3, num_words=5))
print(ldamodel_b2.print_topics(num_topics=3, num_words=5))
print(ldamodel_b3.print_topics(num_topics=3, num_words=5))

[(0, u'0.014*time + 0.009*get + 0.008*work + 0.007*even + 0.007*tri'), (1, u'0.009*updat + 0.006*work + 0.006*even + 0.005*sinc + 0.005*download'), (2, u'0.011*time + 0.008*get + 0.007*work + 0.007*tri + 0.007*updat')]
[(0, u'0.012*updat + 0.010*tri + 0.010*time + 0.010*issu + 0.007*even'), (1, u'0.014*time + 0.014*get + 0.012*work + 0.007*version + 0.007*tri'), (2, u'0.012*time + 0.010*work + 0.010*get + 0.009*even + 0.009*make')]
[(0, u'0.009*la + 0.006*websit + 0.005*store + 0.005*restaur + 0.004*data'), (1, u'0.013*time + 0.012*get + 0.011*work + 0.008*tri + 0.008*even'), (2, u'0.010*time + 0.009*go + 0.008*even + 0.007*need + 0.007*im')]


For the bad app, from the result, we can see most topics include the word "time". We can refer that customer are not satisfied for the using fluency of these apps. And for the updated version of these apps, customers' downloading experience maybe not very good. Meanwhile, compared with website, apps maybe designed not that good. 
